# DB Schema
<img src="./pics/schema.png" width=800>

In [18]:
import neo4j
from neo4j import GraphDatabase

# Establishing connection with remote db
URI = "neo4j+s://abfc122f.databases.neo4j.io"
AUTH = ("neo4j", "Dp5K2MaWGjFA61jgXL5aO3YXEVvjuREXrAQDSVrqyqA")

driver = GraphDatabase.driver(URI, auth=AUTH)
driver.verify_connectivity()

In [42]:
# Creating a user node
query = """
CREATE (u:User {user_id: 123456, display_name: 'Rohit Vernekar'}) 
RETURN u.display_name
"""

driver.execute_query(query, result_transformer_=neo4j.Result.to_df)


,u.display_name
0,Rohit Vernekar


In [43]:
# Updating the user node
query = """
MATCH (u:User {display_name: 'Rohit Vernekar'}) SET u.display_name = 'Rohit V' 
RETURN u.display_name
"""

driver.execute_query(query, result_transformer_=neo4j.Result.to_df)


,u.display_name
0,Rohit V


In [44]:
# Deleting the user node
query = "MATCH (u:User {user_id: 123456}) DELETE u"

driver.execute_query(query, result_transformer_=neo4j.Result.to_df)

""


## Viewing all answers provided by uuid 6077914 as graph
<img src="./pics/query1.png" width=1000>

In [45]:
# Query to get all answers records provided by uuid 6077914
query = """
MATCH (u:User {uuid: 6077914})-[p:PROVIDED]->(a:Answer)
RETURN a
"""

records = driver.execute_query(query, result_transformer_=neo4j.Result.to_df)
records.head()

,a
0,"(score, is_accepted, body_markdown, link, titl..."
1,"(score, is_accepted, body_markdown, link, titl..."
2,"(score, is_accepted, body_markdown, link, titl..."
3,"(score, is_accepted, body_markdown, link, titl..."
4,"(score, is_accepted, body_markdown, link, titl..."


In [46]:
# Query to get all answers body_markdown provided by uuid 6077914
query = """
MATCH (u:User {uuid: 6077914})-[p:PROVIDED]->(a:Answer)
RETURN a.body_markdown
"""

answers = driver.execute_query(query, result_transformer_=neo4j.Result.to_df)
answers.head()

,a.body_markdown
0,**Merge** requires a field(s) which you need t...
1,"First, find all the distinct persons at level ..."
2,If you need parents and grandparents in one li...
3,You can check the equality of the relationship...
4,You can use `IN` to check if a value is presen...


In [47]:
# Query to get see how many questions a user asked
query = """
MATCH (u:User)-[:ASKED]->(q:Question) 
RETURN u.display_name, count(*) AS questions
"""

records = driver.execute_query(query, result_transformer_=neo4j.Result.to_df)
records.head()

,u.display_name,questions
0,schernichkin,1
1,Damian Grzanka,3
2,sandrooco,1
3,Bobis,1
4,wanderingcatto,3


In [48]:
# Query to get top 10 users who asked the most questions
query = """
MATCH (u:User)-[:ASKED]->(q:Question)
RETURN u.display_name, count(*) AS questions
ORDER by questions DESC
LIMIT 10
"""

driver.execute_query(query, result_transformer_=neo4j.Result.to_df)

,u.display_name,questions
0,A. L,13
1,Matt Larson,11
2,Sama,11
3,sten,11
4,jos97,10
5,Aerodynamika,8
6,Simon,8
7,Ilnaz Nikseresht,8
8,marlon,7
9,ck22,7


In [49]:
# Query to get top 10 users who answered the most questions
query = """
MATCH (u:User)-[:PROVIDED]->(a:Answer)
RETURN u.display_name, COUNT(a) AS answers
ORDER BY answers DESC
LIMIT 10
"""

driver.execute_query(query, result_transformer_=neo4j.Result.to_df)

,u.display_name,answers
0,cybersam,211
1,Graphileon,120
2,Tomaž Bratanič,99
3,jose_bacoy,94
4,InverseFalcon,90
5,Christophe Willemsen,43
6,fbiville,32
7,Luanne,27
8,Rajendra Kadam,18
9,Brakebein,17


## Finding shortest between "A. L" who asked most questions and "cybersam" who provided most answers
<img src="./pics/query2.png" width=1000>

## Finding all paths that connect user "A. L" and "cybersam"
<img src="./pics/query3.png" width=1000>

In [50]:
# Query to all the unanswered questions
query = """
MATCH (q:Question)
WHERE NOT (q)<-[:ANSWERED]-()
RETURN q.link, q.body_markdown
"""

questions = driver.execute_query(query, result_transformer_=neo4j.Result.to_df)
questions.head()

,q.link,q.body_markdown
0,https://stackoverflow.com/q/65697147,I have a dataset in java which contains an &qu...
1,https://stackoverflow.com/q/65694121,Given the schema\r\n\r\n```graphql\r\ntype Use...
2,https://stackoverflow.com/q/65691774,I am having trouble updating nodes in a neo4j-...
3,https://stackoverflow.com/q/65690692,My graph has the following entities:\r\n\r\n- ...
4,https://stackoverflow.com/q/65690563,Well first thing this is my code\r\n\r\n LO...


In [51]:
# Query to see the top 10 Tags with most unanswered questions
query = """
MATCH (q:Question)-[:TAGGED]->(t:Tag)
WHERE NOT (q)<-[:ANSWERED]-()
RETURN t.name as tag, count(q) AS questions
ORDER BY questions DESC LIMIT 10;
"""

driver.execute_query(query, result_transformer_=neo4j.Result.to_df)

,tag,questions
0,neo4j,420
1,cypher,186
2,java,41
3,python,33
4,neo4j-apoc,33
5,spring-boot,24
6,graph-databases,21
7,spring,19
8,graphql,18
9,spring-data-neo4j,17
